<a href="https://colab.research.google.com/github/rjatelo30/Final-ML-Task/blob/main/CNN_to_classify_circle_rectangles_and_squares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compiled By:
Ryan Jatelo 110703
Michael Owade 113612

In [ ]:
pip install keras-tuner --upgrade

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPool2D, Dropout
from kerastuner.tuners import RandomSearch

# Import dataset from tensorflow

In [ ]:
data_train = tfds.load(name = "rock_paper_scissors", split="train")
data_test = tfds.load(name = "rock_paper_scissors", split="test")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete1KOSHB/rock_paper_scissors-train.tfrecord


  0%|          | 0/2520 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete1KOSHB/rock_paper_scissors-test.tfrecord


  0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


Previewing the dataset

In [ ]:
data_train

<PrefetchDataset shapes: {image: (300, 300, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [ ]:
data_test

<PrefetchDataset shapes: {image: (300, 300, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>

# Convert the dataset to numpy format

In [ ]:
#Create and store the dataset in a numpy array and remove the color channel

img_train = np.array([example['image'].numpy()[:,:,0] for example in data_train])
labels_train = np.array([example['label'].numpy() for example in data_train])

img_test = np.array([example['image'].numpy()[:,:,0] for example in data_test])
labels_test = np.array([example['label'].numpy() for example in data_test])

In [ ]:
#Reshape the dataset

img_train = img_train.reshape(2520, 300, 300, 1)
img_train.shape 

(2520, 300, 300, 1)

In [ ]:
img_test = img_test.reshape(372, 300, 300, 1)
img_test.shape 

(372, 300, 300, 1)

In [ ]:
#Normalize the values to 0 to 1 instead of 0 to 255

img_train = img_train.astype('float32')
img_test = img_test.astype('float32')

img_train = img_train / 255
img_test = img_test / 255

# Build a CNN with a sequential model

In [ ]:
model = Sequential()
#The size of the images are large for having basic shapes. We shall reduce the resolution.
model.add(AveragePooling2D(6, 3, input_shape=(300, 300, 1))) #Scales the images down to 100 x 100
model.add(Conv2D(128, 3, activation = 'relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2, 2)) #further downsampling
model.add(Dropout(0.5)) #randomly cut out connections by half because of the small dataset provided
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

model.fit(img_train, labels_train, epochs = 50)

Epoch 1/50
79/79 [==============================] - 11s 114ms/step - loss: 0.8752 - accuracy: 0.7476
Epoch 2/50
79/79 [==============================] - 8s 101ms/step - loss: 0.1037 - accuracy: 0.9734
Epoch 3/50
79/79 [==============================] - 8s 101ms/step - loss: 0.0122 - accuracy: 0.9984
Epoch 4/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0249 - accuracy: 0.9937
Epoch 5/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0203 - accuracy: 0.9956
Epoch 6/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0137 - accuracy: 0.9964
Epoch 7/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0048 - accuracy: 0.9992
Epoch 8/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 9/50
79/79 [==============================] - 8s 100ms/step - loss: 0.0034 - accuracy: 0.9992
Epoch 10/50
79/79 [==============================] - 8s 101ms/step - loss: 0.0035 - accuracy: 0.999

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d_1 (Average (None, 99, 99, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 97, 97, 128)       1280      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 95, 95, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 47, 47, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 47, 47, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 141376)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)              

In [ ]:
model.evaluate(img_test, labels_test)

12/12 [==============================] - 1s 85ms/step - loss: 3.4918 - accuracy: 0.7151


[3.4918344020843506, 0.7150537371635437]

# Tuning the hyperparameters

We will use Keras Tuner to choose the best hyper parameters.

In [ ]:
def build_model(hp):
  model_1 = Sequential()
  model_1.add(AveragePooling2D(6, 3, input_shape=(300, 300, 1))) 
  model_1.add(Conv2D(128, 3, activation = 'relu'))
  model_1.add(Conv2D(64, 3, activation='relu'))
  model_1.add(MaxPool2D(2, 2)) 
  model_1.add(Dropout(0.5)) 
  model_1.add(Flatten())
  model_1.add(Dense(hp.Choice("Dlayer", [256, 512]), activation='relu'))
  model_1.add(Dense(3, activation='softmax'))  

  model_1.compile(optimizer = 'Adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

  return model_1

tune = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 7
)

tune.search(img_train, labels_train, validation_data = (img_test, labels_test), epochs = 50)

Trial 2 Complete [00h 07m 23s]
val_accuracy: 0.7177419066429138

Best val_accuracy So Far: 0.7553763389587402
Total elapsed time: 00h 16m 46s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
